In [5]:
from selenium import webdriver

In [6]:
browser = webdriver.Chrome('chromedriver.exe')

In [7]:
browser.get('https://scraping-for-beginner.herokuapp.com/ranking/')

In [10]:
# まずは1つの大きな塊(ランキング1つ1つの囲うタグ)に着目し、それを取得する
# いきなり全体に適応するのではなく、1つのランキングの取得し、そこから目的のデータが取得できるかを確認する
elem_rankingBox = browser.find_element_by_class_name('u_areaListRankingBox')
elem_rankingBox

<selenium.webdriver.remote.webelement.WebElement (session="2d7af80d9af8b894b05a290ec277ece5", element="7042e6ea-7f4a-421b-8745-ffa6a45d9414")>

In [11]:
# class='u_areaListRankingBox'の中身が全て取り出されていることを確認する
# ココからどうやって目的のデータを取り出すかを考える
elem_rankingBox.text

'1\n観光地 1\n4.7\n楽しさ\n4.6\nとてもエンジョイした\n人混みの多さ\n4.5\n空いていた\n景色\n4.9\n大自然を感じることができた\nアクセス\n4.2\n飛行機で1時間ほどで着きました'

In [15]:
# 取り出したelem_rankingBoxから更にfind_～メソッドを使用して絞り込みを行う
elem_title = elem_rankingBox.find_element_by_class_name('u_title')
title = elem_title.text

In [16]:
title.split('\n')

['1', '観光地 1']

In [17]:
title.split('\n')[1]

'観光地 1'

In [18]:
elem_rankingBox = browser.find_element_by_class_name('u_areaListRankingBox')
elem_rankingBox

<selenium.webdriver.remote.webelement.WebElement (session="2d7af80d9af8b894b05a290ec277ece5", element="7042e6ea-7f4a-421b-8745-ffa6a45d9414")>

In [19]:
elem_comprehensiveEvaluation = elem_rankingBox.find_element_by_class_name('u_rankBox')
elem_comprehensiveEvaluation

<selenium.webdriver.remote.webelement.WebElement (session="2d7af80d9af8b894b05a290ec277ece5", element="d50347dc-6baf-49de-9f60-80f0358ddd3c")>

In [20]:
elem_comprehensiveEvaluation.find_element_by_class_name('evaluateNumber')
elem_comprehensiveEvaluation.text

'4.7'

In [22]:
# 総合評価はclass='evaluateNumber'の最初の1つ目なので、以下の処理でも取得可能
elem_rankingBox = browser.find_element_by_class_name('u_areaListRankingBox')
elem_rankingBox.find_element_by_class_name('evaluateNumber')
elem_comprehensiveEvaluation.text

'4.7'

In [23]:
# その他に関しては、工夫が必要
elem_rankingBox = browser.find_element_by_class_name('u_areaListRankingBox')
elem_category = elem_rankingBox.find_element_by_class_name('u_categoryTipsItem')

In [25]:
evaluates = []
elem_evaluates = elem_category.find_elements_by_class_name('evaluateNumber')

for evaluate in elem_evaluates:
    evaluates.append(evaluate.text)

evaluates

['4.6', '4.5', '4.9', '4.2']

In [28]:
# for文を使用して、一括してデータ収集を行う
elems_rankingBox = browser.find_elements_by_class_name('u_areaListRankingBox')

In [29]:
# class='u_areaListRankingBox'の塊がいくつあるかを確認
len(elems_rankingBox)

10

In [31]:
# すべての観光地名を取得し、リストtitlesに格納する
titles = []

for elem_rankingBox in elems_rankingBox:
    elem_title = elem_rankingBox.find_element_by_class_name('u_title')
    title = elem_title.text.split('\n')[1]
    titles.append(title)
    
titles

['観光地 1',
 '観光地 2',
 '観光地 3',
 '観光地 4',
 '観光地 5',
 '観光地 6',
 '観光地 7',
 '観光地 8',
 '観光地 9',
 '観光地 10']

In [41]:
# 全ての総合評価を取得し、リストcomprehensiveEvaluationsに格納する
elems_rankingBox = browser.find_elements_by_class_name('u_areaListRankingBox')

comprehensiveEvaluations = []

for elem_rankingBox in elems_rankingBox:
    evaluate = elem_rankingBox.find_element_by_class_name('evaluateNumber')
    evaluate = float(evaluate.text)
    comprehensiveEvaluations.append(evaluate)

comprehensiveEvaluations

[4.7, 4.7, 4.6, 4.5, 4.5, 4.4, 4.3, 4.3, 4.2, 4.1]

In [42]:
# 全ての各カテゴリの評価を取得し、リストbyCategoryEvaluatesに格納する
elems_rankingBox = browser.find_elements_by_class_name('u_areaListRankingBox')

byCategoryEvaluates = []

for elem_rankingBox in elems_rankingBox:
    evaluates = []
    elem_categoryTipsItem = elem_rankingBox.find_element_by_class_name('u_categoryTipsItem')
    byCategoryEvaluate = elem_categoryTipsItem.find_elements_by_class_name('evaluateNumber')
    for evaluate in byCategoryEvaluate:
        evaluates.append(float(evaluate.text))
    byCategoryEvaluates.append(evaluates)

byCategoryEvaluates

[[4.6, 4.5, 4.9, 4.2],
 [4.6, 4.5, 4.9, 4.2],
 [4.5, 4.4, 4.8, 4.1],
 [4.4, 4.4, 4.8, 4.0],
 [4.4, 4.3, 4.7, 4.0],
 [4.3, 4.3, 4.7, 3.9],
 [4.2, 4.2, 4.6, 3.8],
 [4.2, 4.2, 4.6, 3.8],
 [4.1, 4.1, 4.5, 3.7],
 [4.0, 4.1, 4.4, 3.6]]

In [43]:
import pandas as pd

In [45]:
df = pd.DataFrame()

In [46]:
df['観光地名'] = titles
df['総合評価'] = comprehensiveEvaluations

In [49]:
df_categories = pd.DataFrame(byCategoryEvaluates)
df_categories.columns = ['楽しさ', '人混みの多さ', '景色', 'アクセス']
df_categories

,楽しさ,人混みの多さ,景色,アクセス
0,4.6,4.5,4.9,4.2
1,4.6,4.5,4.9,4.2
2,4.5,4.4,4.8,4.1
3,4.4,4.4,4.8,4.0
4,4.4,4.3,4.7,4.0
5,4.3,4.3,4.7,3.9
6,4.2,4.2,4.6,3.8
7,4.2,4.2,4.6,3.8
8,4.1,4.1,4.5,3.7
9,4.0,4.1,4.4,3.6


In [50]:
df = pd.concat([df, df_categories], axis=1)
df

,観光地名,総合評価,楽しさ,人混みの多さ,景色,アクセス
0,観光地 1,4.7,4.6,4.5,4.9,4.2
1,観光地 2,4.7,4.6,4.5,4.9,4.2
2,観光地 3,4.6,4.5,4.4,4.8,4.1
3,観光地 4,4.5,4.4,4.4,4.8,4.0
4,観光地 5,4.5,4.4,4.3,4.7,4.0
5,観光地 6,4.4,4.3,4.3,4.7,3.9
6,観光地 7,4.3,4.2,4.2,4.6,3.8
7,観光地 8,4.3,4.2,4.2,4.6,3.8
8,観光地 9,4.2,4.1,4.1,4.5,3.7
9,観光地 10,4.1,4.0,4.1,4.4,3.6


In [51]:
df.to_csv('観光地情報.csv', index=False)

In [52]:
# ページネーション対策
# URLの変化に注目、これをfor文で回す
i = 1
'https://scraping-for-beginner.herokuapp.com/ranking/?ranks={}'.format(i)

'https://scraping-for-beginner.herokuapp.com/ranking/?ranks=1'

In [56]:
titles = []
comprehensiveEvaluations = []
byCategoryEvaluates = []

for page in range(1, 4):
    
    # 目的のページに遷移させる
    url = 'https://scraping-for-beginner.herokuapp.com/ranking/?page={}'.format(page)
    browser.get(url)
    
    # 1ページ毎にランキングの1かたまりの要素をリストで取得
    elems_rankingBox = browser.find_elements_by_class_name('u_areaListRankingBox')
    
    # すべての観光地名を取得し、リストtitlesに格納する
    for elem_rankingBox in elems_rankingBox:
        elem_title = elem_rankingBox.find_element_by_class_name('u_title')
        title = elem_title.text.split('\n')[1]
        titles.append(title)
        
    # 全ての総合評価を取得し、リストcomprehensiveEvaluationsに格納する
    for elem_rankingBox in elems_rankingBox:
        evaluate = elem_rankingBox.find_element_by_class_name('evaluateNumber')
        evaluate = float(evaluate.text)
        comprehensiveEvaluations.append(evaluate)
        
    # 全ての各カテゴリの評価を取得し、リストbyCategoryEvaluatesに格納する
    for elem_rankingBox in elems_rankingBox:
        evaluates = []
        elem_categoryTipsItem = elem_rankingBox.find_element_by_class_name('u_categoryTipsItem')
        byCategoryEvaluate = elem_categoryTipsItem.find_elements_by_class_name('evaluateNumber')
        for evaluate in byCategoryEvaluate:
            evaluates.append(float(evaluate.text))
        byCategoryEvaluates.append(evaluates)


In [57]:
titles

['観光地 1',
 '観光地 2',
 '観光地 3',
 '観光地 4',
 '観光地 5',
 '観光地 6',
 '観光地 7',
 '観光地 8',
 '観光地 9',
 '観光地 10',
 '観光地 11',
 '観光地 12',
 '観光地 13',
 '観光地 14',
 '観光地 15',
 '観光地 16',
 '観光地 17',
 '観光地 18',
 '観光地 19',
 '観光地 20',
 '観光地 21',
 '観光地 22',
 '観光地 23',
 '観光地 24',
 '観光地 25',
 '観光地 26',
 '観光地 27',
 '観光地 28',
 '観光地 29',
 '観光地 30']

In [58]:
comprehensiveEvaluations

[4.7,
 4.7,
 4.6,
 4.5,
 4.5,
 4.4,
 4.3,
 4.3,
 4.2,
 4.1,
 4.1,
 4.0,
 3.9,
 3.9,
 3.8,
 3.7,
 3.7,
 3.6,
 3.5,
 3.5,
 3.4,
 3.3,
 3.3,
 3.2,
 3.1,
 3.1,
 3.0,
 2.9,
 2.9,
 2.8]

In [59]:
byCategoryEvaluates

[[4.6, 4.5, 4.9, 4.2],
 [4.6, 4.5, 4.9, 4.2],
 [4.5, 4.4, 4.8, 4.1],
 [4.4, 4.4, 4.8, 4.0],
 [4.4, 4.3, 4.7, 4.0],
 [4.3, 4.3, 4.7, 3.9],
 [4.2, 4.2, 4.6, 3.8],
 [4.2, 4.2, 4.6, 3.8],
 [4.1, 4.1, 4.5, 3.7],
 [4.0, 4.1, 4.4, 3.6],
 [4.0, 4.0, 4.4, 3.6],
 [3.9, 4.0, 4.3, 3.5],
 [3.8, 3.9, 4.3, 3.4],
 [3.8, 3.9, 4.2, 3.4],
 [3.7, 3.8, 4.2, 3.3],
 [3.6, 3.8, 4.1, 3.2],
 [3.6, 3.7, 4.1, 3.2],
 [3.5, 3.7, 4.0, 3.1],
 [3.4, 3.6, 3.9, 3.0],
 [3.4, 3.6, 3.9, 3.0],
 [3.3, 3.5, 3.8, 2.9],
 [3.2, 3.5, 3.8, 2.8],
 [3.2, 3.4, 3.7, 2.8],
 [3.1, 3.4, 3.7, 2.7],
 [3.0, 3.3, 3.6, 2.6],
 [3.0, 3.3, 3.6, 2.6],
 [2.9, 3.2, 3.5, 2.5],
 [2.8, 3.2, 3.4, 2.4],
 [2.8, 3.1, 3.4, 2.4],
 [2.7, 3.1, 3.3, 2.3]]

In [60]:
df = pd.DataFrame()
df['観光地名'] = titles
df['総合評価'] = comprehensiveEvaluations

df_categories = pd.DataFrame(byCategoryEvaluates)
df_categories.columns = ['楽しさ', '人混みの多さ', '景色', 'アクセス']
df = pd.concat([df, df_categories], axis=1)
df

,観光地名,総合評価,楽しさ,人混みの多さ,景色,アクセス
0,観光地 1,4.7,4.6,4.5,4.9,4.2
1,観光地 2,4.7,4.6,4.5,4.9,4.2
2,観光地 3,4.6,4.5,4.4,4.8,4.1
3,観光地 4,4.5,4.4,4.4,4.8,4.0
4,観光地 5,4.5,4.4,4.3,4.7,4.0
5,観光地 6,4.4,4.3,4.3,4.7,3.9
6,観光地 7,4.3,4.2,4.2,4.6,3.8
7,観光地 8,4.3,4.2,4.2,4.6,3.8
8,観光地 9,4.2,4.1,4.1,4.5,3.7
9,観光地 10,4.1,4.0,4.1,4.4,3.6


In [61]:
df.to_csv('観光地情報.csv', index=False)